In [9]:
import os
import pandas as pd
from dotenv import load_dotenv

load_dotenv()

True

In [10]:
df = pd.read_excel("../data/MPI_Directory_by_Establishment_Number.xlsx")

/opt/conda/lib/python3.11/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


In [11]:
df.head(5)

,EstNumber,Establishment ID,Company,Street,City,State,Zip,Phone,GrantDate,Activities,DBAs,LatestMPIActiveDate
0,G1028,195,Papetti's Hygrade Egg Products Inc.,One Papetti Plaza,Elizabeth,NJ,7206,(908) 282-7900,12/05/2019,Egg Product,NaN,2023-09-18
1,G1105,126734,"American Egg Products, LLC",375 Pierce Industrial Blvd.,Blackshear,GA,31516,(912) 449-5700,09/22/2021,Egg Product,NaN,2023-09-18
2,G1126A,126639,Shepherds Processed Eggs,3502 West 6400 South,Spanish Fork,UT,84660,(801) 798-2593,01/28/2022,Egg Product,NaN,2023-09-18
3,G1126B,6164012,Shepherds Processed Eggs,3465 West 6400 South,Spanish Fork,UT,84660,NaN,01/28/2022,Egg Product,NaN,2023-09-18
4,G1141,8204,"Wabash Valley Produce, Inc.",501 S. Chestnut,Farina,IL,62838,NaN,03/04/2022,Egg Product,NaN,2023-09-18


In [12]:
BASE_URL = "https://api.mapbox.com/geocoding/v5/mapbox.places/"
MAPBOX_ACCESS_TOKEN = os.environ["MAPBOX_ACCESS_TOKEN"]

In [13]:
import requests

In [14]:
geocoded_rows = []
for idx, row in df[:5].iterrows():
    url = (
        BASE_URL
        + row.Street
        + " "
        + row.City
        + " "
        + row.State
        + " "
        + str(row.Zip)
        + ".json?access_token="
        + MAPBOX_ACCESS_TOKEN
    )
    r = requests.get(url)
    lng, lat = r.json()["features"][0]["center"]
    row['lat'] = lat
    row['lng'] = lng
    geocoded_rows.append(row)

In [18]:
df.tail(150).to_json("../data/")

,EstNumber,Establishment ID,Company,Street,City,State,Zip,Phone,GrantDate,Activities,DBAs,LatestMPIActiveDate
6769,V46744,6164686,"Lineage Logistics, LLC",3269 E. 44th Street,Vernon,CA,90058,(323) 583-3163,07/17/2020,"Certification - Export, Identification - Meat,...",NaN,2023-09-18
6770,V46745,6164688,"Lineage Logistics, LLC",2825 E. 44th Street,Vernon,CA,90058,(323) 583-3163,07/17/2020,"Certification - Export, Identification - Meat,...",NaN,2023-09-18
6771,V46746,6164689,"Lineage Logistics, LLC",3211 E. 44th Street,Vernon,CA,90058,(323) 583-3163,07/17/2020,"Certification - Export, Certification - Trichi...",NaN,2023-09-18
6772,V46749,6164692,Channel Island Logistics Inc.,1819 E. Pacific Coast Highway,Wilmington,CA,90744,(424) 264-2753,03/02/2023,Certification - Export,NaN,2023-09-18
6773,V46763,6164715,Star Warehouse,7007 NW 37th Avenue,Miami,FL,33047,(561) 477-2998,07/15/2019,"Certification - Export, Identification - Meat,...",NaN,2023-09-18
...,...,...,...,...,...,...,...,...,...,...,...,...
6914,V920,6166803,Exel Inc DBA DHL Supply Chain,1005 Railhead Drive,Fort Worth,TX,76177,(630) 974-9220,08/14/2022,"Certification - Export, Identification - Meat,...",NaN,2023-09-18
6915,V946,123190,International Exports Inc.,10820 Train Court,Houston,TX,77041,(713) 849-2299,12/08/2011,"Certification - Export, Identification - Meat,...",NaN,2023-09-18
6916,V9553A,6162923,Godshall's Quality Meats Inc.,743 Hagey Center Drive,Souderton,PA,18964,(215) 256-8867,07/17/2020,"Certification - Export, Identification - Meat,...",NaN,2023-09-18
6917,V991,123429,United Natural Foods Inc.,6272 McIntosh Road,Sarasota,FL,34238,(941) 925-6600,02/19/2021,"Identification - Meat, Identification - Poultry",NaN,2023-09-18


In [15]:
df_geocoded = pd.DataFrame(geocoded_rows)
df_geocoded.head(2)

,EstNumber,Establishment ID,Company,Street,City,State,Zip,Phone,GrantDate,Activities,DBAs,LatestMPIActiveDate,lat,lng
0,G1028,195,Papetti's Hygrade Egg Products Inc.,One Papetti Plaza,Elizabeth,NJ,7206,(908) 282-7900,12/05/2019,Egg Product,NaN,2023-09-18,40.657042,-74.190242
1,G1105,126734,"American Egg Products, LLC",375 Pierce Industrial Blvd.,Blackshear,GA,31516,(912) 449-5700,09/22/2021,Egg Product,NaN,2023-09-18,31.304123,-82.232523


In [16]:
df_geocoded.to_csv("../data/fsis-geocoded.csv")